In [25]:
import pandas as pd

import gensim
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim import corpora, models
from gensim.models import LdaMulticore
from gensim.models import Word2Vec
from gensim.corpora import Dictionary, MmCorpus

import numpy as np
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.layers as layers
from text_loader import TextLoader

In [2]:
#!pip install TextLoader

## 1단계 : 데이터 준비

팀원이 미리 전처리하여준 데이터를 이용하였다.

In [3]:
data = pd.read_csv('../datasets/1990-2000_R_B_Soul.tsv', sep='\t', encoding = "mbcs")
data = data.append(pd.read_csv('../datasets/1990-2000_댄스.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/1990-2000_랩_힙합.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/1990-2000_록_메탈.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/1990-2000_발라드.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/1990-2000_인디음악.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/1990-2000_포크_블루스.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/2001-2010_R_B_Soul.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/2001-2010_댄스.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/2001-2010_랩_힙합.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/2001-2010_록_메탈.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/2001-2010_발라드.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/2001-2010_인디음악.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/2001-2010_포크_블루스.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/2011-2019_R_B_Soul.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/2011-2019_댄스.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/2011-2019_랩_힙합.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/2011-2019_록_메탈.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/2011-2019_발라드.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/2011-2019_인디음악.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data = data.append(pd.read_csv('../datasets/2011-2019_포크_블루스.tsv', sep='\t', encoding = "mbcs"), ignore_index=True)
data

,년도,제목,장르,가사
0,1995,이 밤의 끝을 잡고,R&B/Soul,다신 널 볼 수 없겠지 나의 입술이 너의 하얀 어깨를 감싸 안으며 그렇게 우린 이 ...
1,1995,나만의 친구,R&B/Soul,늦어버린 아침에 난 거리에서 허전해진 나의 마음 달래려 했어 그래 난 내 주변사람 ...
2,1995,잠든 널 포켓속에,R&B/Soul,날 따르지 않을거란 걸 알기에 오늘밤은 잠든 니가 너무도 좋아 이대로 두고 가면 사...
3,1995,아끼지 못했던 사랑,R&B/Soul,Listen girl 다시 시작한다고 해도 처음으로 갈 순 없겠지 또 다른 후회속에...
4,1996,천생연분,R&B/Soul,너무 너무 예쁘다고 해도 너를 떠올리며 거절했지만 이번 한 번 뿐이라는걸 맹세해 약...
...,...,...,...,...
6063,2018,매트리스,포크/블루스,오늘밤 너는 나와 이불 속에 들어가 아무것도 하지 말고 그냥 바라보다가 웃음을 참지...
6064,2019,그러나,포크/블루스,오랜만에 나를 마주치는 어느정도 가까운 사람들이 나에게 다가와 인사하며 꺼내는 너의...
6065,2019,"나만, 봄",포크/블루스,안돼 그만둬 거기까지 해 더 다가가면 너 정신 못 차려 안돼 그만해 꽃은 넣어둬 그...
6066,2019,별 보러 갈래?,포크/블루스,maybe it's like a dream I see the stars over m...


본래 시대와 장르를 따로하여 학습을 해야하나 1차적으로 결과가 궁금하여 모든 데이터를 합하여 테스트를 진행해 보았다.

In [4]:
%%time
USE_PREMADE_TEXT = False

text_filepath = 'word2vec/all_lyrics_text.txt'
if not USE_PREMADE_TEXT:
    with open(text_filepath, 'w', encoding='utf-8') as f:
        for lyrics in data.가사.values:
            if pd.isnull(lyrics):
                continue
            f.write(lyrics + '\n')
else:
    assert path.exists(text_filepath)

Wall time: 55.9 ms


In [5]:
%%time
USE_PREMADE_BIGRAM_MODEL = False

all_bigram_model_filepath = 'word2vec/all_bigram_model'
all_sentences_normalized_filepath = 'word2vec/all_lyrics_text.txt'

all_unigram_sentences = LineSentence(all_sentences_normalized_filepath)

if not USE_PREMADE_BIGRAM_MODEL:    
    
    all_bigram_model = Phrases(all_unigram_sentences) #phrase냐 아니냐를 판단해줌
    all_bigram_model.save(all_bigram_model_filepath)
    
else:
    all_bigram_model = Phrases.load(all_bigram_model_filepath)

Wall time: 3.35 s


In [6]:
%%time
USE_PREMADE_BIGRAM_SENTENCES = False

all_bigram_sentences_filepath = 'word2vec/all_sentences_for_word2vec.txt'

if not USE_PREMADE_BIGRAM_SENTENCES:
    
    with open(all_bigram_sentences_filepath, 'w', encoding='utf-8') as f:
        for unigram_sentence in all_unigram_sentences:
            all_bigram_sentence = all_bigram_model[unigram_sentence]
            f.write(' '.join(all_bigram_sentence) + '\n')
else:
    assert path.exists(all_bigram_sentences_filepath)

Wall time: 4.99 s


In [7]:
%%time
USE_PREMADE_WORD2VEC = False

all2vec_filepath = 'word2vec/all_word2vec_model'

if not USE_PREMADE_WORD2VEC:
    
    lyrics_for_word2vec = LineSentence(all_bigram_sentences_filepath)

    all2vec = Word2Vec(lyrics_for_word2vec, size=100, window=5, min_count=1, sg=1)
    # sg=0 cbow 1=Skip-Gram Model
    # 100차원으로 가져옴 / 보통 20~100 정도
    # window = 5 앞 5개, 뒤 5개 단어를 보겠다는 뜻
    # window size 작을수록 문법적인 의미가 너무 중요해짐, 클수록 주제 지향적으로 문맥적인 정보를 많이 담게 됨    
    for _ in range(9):
        all2vec.train(lyrics_for_word2vec, total_examples=789847, epochs=1)
        # 9139곡 가사에 총 789847 문장이 있다는 것 명시
        
    all2vec.save(all2vec_filepath)
else:
    all2vec = Word2Vec.load(all2vec_filepath)
all2vec.init_sims()

Wall time: 1min 7s


In [8]:
all2vec_filepath = 'word2vec/all_word2vec_model'
all2vec = Word2Vec.load(all2vec_filepath)

In [9]:
all2vec.most_similar(positive=['사랑'], topn=5)

D:\Users\User\AppData\Local\Continuum\anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('묻었어', 0.7488182783126831),
 ('스며들면', 0.7111870050430298),
 ('고마워했던', 0.6880630254745483),
 ('직선들', 0.6850664019584656),
 ('도대체?안지워져', 0.6810793876647949)]

In [10]:
all2vec.most_similar(positive=['첫번째'], topn=5)

D:\Users\User\AppData\Local\Continuum\anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('여자잖아', 0.6793165802955627),
 ('보았고', 0.6435946822166443),
 ('통로로', 0.6228785514831543),
 ('수없이_그대만을', 0.6069067716598511),
 ('실연의', 0.5952743291854858)]

In [11]:
a = pd.DataFrame(all2vec.wv.index2word)
a

,0
0,난
1,내
2,you
3,날
4,널
...,...
98583,가는곳으로
98584,오직나는
98585,즐거웠잖아
98586,으음


In [12]:
words = []
X = []
for i in range(98588):
    words.append(a[0][i])
    i+=1
    
for i in words:
    X.append(all2vec.wv[i])

In [13]:
X2 = pd.DataFrame(X, index=words)
X2

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
난,0.324973,0.560413,-0.111913,-0.317721,0.130772,0.811746,0.725740,0.868835,-0.013499,-0.436225,...,-0.305045,0.035990,0.350707,0.049877,0.579402,-0.066479,-0.058379,-0.241697,0.244572,0.204359
내,-0.252698,0.181000,0.535097,0.118259,-0.284157,0.487642,0.877686,0.054122,0.402573,0.005149,...,-0.108613,0.138564,-0.383185,-0.101908,-0.158791,-0.302360,0.444354,0.285284,0.182249,0.126824
you,0.368872,0.247509,0.400868,0.642247,0.406013,0.832386,0.337338,0.038737,-0.365573,-0.030703,...,0.345163,0.403899,-0.267906,0.391355,-0.091253,0.303192,0.418106,0.102866,-0.278407,-0.077569
날,-0.065219,0.253183,-0.347977,-0.205669,0.268042,0.267922,-0.303527,0.472282,0.449822,-0.003439,...,-0.080445,0.159957,-0.060478,-0.382070,0.293718,-0.192820,-0.321762,-0.238475,0.253597,0.412574
널,0.006007,0.454094,-0.047977,0.123895,0.296453,0.590186,0.079995,0.469293,-0.270900,0.278150,...,0.191817,-0.246357,-0.352866,-0.287637,-0.085968,0.062326,0.173418,-0.550097,0.112606,-0.023478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
가는곳으로,0.089716,-0.166492,-0.011216,0.191420,0.112463,0.317050,0.031168,0.323857,0.235035,-0.064674,...,0.031887,-0.284336,-0.039370,0.147808,0.110808,0.151268,0.225303,0.015159,-0.060313,-0.174639
오직나는,0.013922,-0.250585,0.080971,-0.021951,0.014908,0.178208,0.024611,0.020863,0.169250,0.017469,...,-0.054752,-0.053910,-0.054177,-0.087427,-0.234253,-0.049626,0.223956,-0.160064,-0.003117,-0.128978
즐거웠잖아,0.234306,-0.231585,0.070613,0.178579,0.321745,0.247071,0.151800,0.129247,0.367465,0.195350,...,0.081168,0.053150,-0.151811,-0.006829,-0.096275,-0.086571,0.196243,-0.168063,-0.115017,-0.142452
으음,0.188734,0.134341,-0.094361,0.064645,0.031889,0.256337,-0.003096,0.222790,0.414015,-0.012352,...,0.111815,-0.085342,-0.070342,0.070630,0.056998,-0.022296,-0.150403,-0.205069,0.103018,-0.130270


In [26]:
num_layers  = 3
hidden_size = 512
batch_size  = 200
max_length  = 30
learning_rate = 0.001

loader = TextLoader("sentences_for_word2vec.txt")
vocab_size = len(loader.vocab)

In [27]:
#with tf.device("/gpu:5"):
X = tf.placeholder(tf.float32, [None, max_length, 100])
y = tf.placeholder(tf.int32, [None, max_length]) # 인풋은 워드투벡 아웃풋은 단어?

y_one_hot = tf.one_hot(y, vocab_size)

cells = [rnn.BasicLSTMCell(hidden_size) for _ in range(num_layers)]
cells = rnn.MultiRNNCell(cells, state_is_tuple=True)

initial_state = cells.zero_state(batch_size, tf.float32)
outputs, _ = tf.nn.dynamic_rnn(cells, X, initial_state=initial_state, dtype=tf.float32)

outputs = tf.reshape(outputs, [-1, hidden_size])
logits = layers.linear(outputs, vocab_size)
y_flat = tf.reshape(y_one_hot, [-1, vocab_size])


loss_op = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_flat)
loss_op = tf.reduce_mean(loss_op)

opt = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

y_softmax = tf.nn.softmax(logits)
pred = tf.argmax(y_softmax, axis=1)
pred = tf.reshape(pred, [batch_size, -1])

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
saver = tf.train.Saver()

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
# model_filepath=

with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, "char-rnn") # 

    for step in range(0, 100000):
        X_wv, X_idx, y_wv, y_idx = loader.next_batch(batch_size, max_length)
        loss, _ = sess.run([loss_op, opt], feed_dict={X: X_wv, y: y_idx})

        if (step+1) % 10 == 0:
            print("{:08d} step, loss:{:.4f}".format(step+1, loss))

            random = np.random.randint(0, batch_size)
            results = sess.run(pred, feed_dict={X: X_wv})
            words = [loader.words[word] for word in results[random]]
            print(" ".join(words))

        if (step+1) % 1000 == 0:
            saver.save(sess, "char-rnn_"+str(step+1)) # 원래 checkpoints/

00000010 step, loss:9.7803
내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내
00000020 step, loss:10.2762
you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you
00000030 step, loss:9.1803
내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내
00000040 step, loss:10.8651
난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난
00000050 step, loss:10.8359
내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내
00000060 step, loss:10.0526
난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난
00000070 step, loss:10.6744
내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내 내
00000080 step, loss:9.4935
난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난
00000090 step, loss:9.5311
you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you
00000100 step, loss:10.7935
난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난 난
0000

다 사랑하는 모두 돌아와 me 그랬던_것처럼 아직 괜찮아 수_없는 말고 없어 내_곁에 아직 다 모습 마지막 다 위해 내맘 오직 난 받아 아직 모두 난 난 모두 to_the 하는 난
00000710 step, loss:6.7726
a 모두 거야 그 또 숨이 shake_shake 괜찮아 your want_you in and and 말은 아직 또 나를 내 볼_때 내 모습 a 구멍 워 수_없어 마지막 되기 - 점점 그
00000720 step, loss:7.3747
난 you 누구보다 you 괜찮아 you 모두 you 어떤 처음 You're_my 널_위한 또 다 oh_oh tonight 수_없는 You're_my You're_my 5년간 Zim-zimzalabim_Zim-zim 어떤 네게 널_위한 하지만 처럼 마요 없어 내가_먼저 거야
00000730 step, loss:7.0171
내가_먼저 자 우린 다 이제 널_사랑해 모두 그 거야 다 곁에 없어 내가_먼저 내가_먼저 점점 나를 down 자 또 내 모두 내 받아 다 좋아요 이제는 숨이 처음 또 내
00000740 step, loss:6.7683
오직 어떻게 처럼 나의 그 네게 것이_내겐 네게 말고 겨울로 네게 널_위한 - 수_없는 your 내가_먼저 그건 네게 다시 받아 이젠 a 다 이렇게 네게 말은 오직 말은 your 내가_먼저
00000750 step, loss:7.2024
모두 게 내 생각하며 제발 숨이 말은 거야 a 없어 숨이 점점 내 정말 말고 your 해 처럼 수_없는 la_la 모두 a 널_위한 또 깜짝 사랑의 내 in 남은 수_없는
00000760 step, loss:7.3840
정말 난 이름 오 your 힘 개 개 개 Rally_Rally 그건 개 개 개 거야 당신 개 아무런_말없이 다 깜짝 개 아무런_말없이 다 말을 you girl la_la 난 더 어깨를
00000770 step, loss:6.9901
그건 그건 수가 내가_먼저 Hey_Hey 그건 그건 그건 그건 그건 그건 않아 몰라 남은 그건

00001250 step, loss:5.1309
또 어떤 어느새 그냥 믿고 싶지 is 꽃잎엔 to 것 이_넓은 물든 그냥 너는 바랬던 하지만 진짜 점점 내가_먼저 해도 우린 날에_멈춘 오직_너뿐이라고 해 날에_멈춘 같아요 너 오래_이별하는 없어 아무
00001260 step, loss:5.3669
this now 그대로 말은 잊을_수 고개를 Love 숨이 오 혹시나 내가 너에게 줄_수 널_위한 오직 널_위한 태어난 sorry_but Baby 사랑에 널_보면 내 두_눈이 있는걸 없던 말은 수가_없어 너는 내게 널_위해
00001270 step, loss:4.9859
you 불렀을 수 항상 있게 지금껏 있던 돌아와 you 가버리면 수 언제나 괜찮아 그거 있던 아니면 때까지 말은 수_없어 하지만 수 때 그 원하는 내가 너의 내_곁에 줄게 있던 우리가
00001280 step, loss:5.0561
Rally_Ra Rally_Ra 더 더 더 더 더 더 더 더 더 세상 태어난 다 better_not 그 순간 우린 하나 hey this_is go 널_위한 순_없어 더_큰 이젠 두려워 말아요 때론 태어난
00001290 step, loss:4.7989
너를 몰래 수_없는 내가 아직 싶은 것_같아 자릴 You're_my 싶어 말고 그대는 우 맘이 어떻게 돌아와 지금은 있다면 더 두고 내릴 부디 한동안 부디 어느새 꿀 그랬던 니가 너무 좋아한다는
00001300 step, loss:5.1077
하루종일 평생을 수_없는 있어요 널_위해 and i_love 말고 밤 끝이 내겐 이제 알고 아직도 난 받아 in if 모두 빨리 채 난 이 내가_먼저 어둠을 Love_Fake the bad_men right what
00001310 step, loss:4.7198
that 내_맘을 지금껏 말고 다가와 그래 그렇게 네 믿고 수가 순간 나만 알_수 있게 know 할_수 한마디 말고 그대에게 날 너무나 줘 애써 나를 오직 네가 어디 내_맘이 널 너를_사랑해
00001320 step, loss

00001800 step, loss:3.6925
어디 같이 세상이 내가_먼저 걸렸네 모습을 나는 이름 지금 네가 하는 친한 너도 정말 있죠 I_don't 왜 넌 자꾸 종교, 아늑했던 위에 수_없어 give you everything Baby i'm_so now_We can_fly
00001810 step, loss:3.8253
아름다워 지금껏 부족한 지금껏 나 안으면 나 그저 네 눈빛에 알_것 감정은 꼬마 생각은 줄_알았어 나 가슴에 나 인생을 있다 같아 너의 손을 그대여 네 입술에 조금씩 암것도 할 때면
00001820 step, loss:4.2921
없다고 싶은 부수네 난 같애 노을빛 내게 Style_미인계의 Honeybee, 나는 손을 처음에는 처음에는 어색한 처음에는 그 Style_미인계의 TO 어머_깜짝야 어머_깜짝야 내가 Oi Honeybee, 한동안 이럴 우리들의 모습을 몰래 아직도 혼자
00001830 step, loss:4.4436
마음에 들면 하나같이 있다면 니가_뭔데 가는 또 gonna_back 봐 만날_수 발도 봐 나 아파도_강한 Bo_Peep 아무것도 있었어_사랑이 Everyday,_day, 나 모든_약점들은 나보다_더 내 너무나_사랑해서 TO 학교 눈부셔_눈부셔 그냥 말이 때론 별이
00001840 step, loss:4.4860
내게는 지금 없이_살 계속 이렇게 너를 좀_더 하다 너를 뒤에서 어떤 않는 하하 Bo_Peep 나보다_더 마치고 난 지금 화가 잠들기_전에 어머_깜짝야 TO 째깍째깍_시간이 나보다_더 감정은 어머_깜짝야 어머_깜짝야 전혀 너와의 않아
00001850 step, loss:4.0035
어떤 바라봐 싶었어 듯 처음에는 나보다_더 너를 매었어 널 찾아 너_하나뿐야 날 그날 내_맘속에 사랑해_사랑해 달이 좋으니 봤던 내게만 걸까 그렇게 조금씩 태연하게 넌 You wanna bad_boy tell_tell you wanna
00001860 step, loss:4.0993
오라이_오라이 오라이_오라이 것을 물든 않아 그럴

00002340 step, loss:3.4080
바라보며 적도 직전 너무너무 알았어 너보고 친구들도 깨끗한 직전 직전 연속 그렇게 싶은 있지 말고 나랑 같이 가요 Come_on baby 오직 일 억지로 붙잡고 뭐로 쉬고 차이로 거란 대한 해변가
00002350 step, loss:3.2347
길을 나_변했습니다 개 Rally_Ra 혹시 책을 Rally_Rally 이미 바꿨어 혹시 니_생각이 날까 걸린 개 하지만 책을 아직 줄_몰라서 새 혹시 그리워서 LOVE Rally_Ra 종일 Bo_Peep 아직 앓고_있었다고 우린 아직 사랑
00002360 step, loss:3.5415
옆에서 한번도 안겨 All_night long 오늘 그대와 단_둘이 함께 사랑한다고 천사들의 스쳐가는 하나 나의 입술 된달지어도 사랑을 초월한거야 너는 그냥 내 마음속 사랑 노래를 지난날의 돼 도와줘 너에게 너와의 열어줘
00002370 step, loss:2.8765
이 솔직한 그리움 대한 몰라요 내란 깊숙이 Tree 기억할게 부족한 내 대체_뭐해 지워지지 그곳에 내 많이_사랑한 지워지지 종이가 직전 그 뒤에서 기분을 지금껏 빛나는 그 모습에 나도 따라 방식일 그대
00002380 step, loss:2.7811
and I 서로 돌아보지마 널 감당해볼게 배가 더 돌아올까 싫어 이건 어때 My babe 조금 남자가 너무 뒤에서 않게 간절히 때 늘 잠 준비가 It's_time 근데 나도_모르게 자꾸 내곁에 돼
00002390 step, loss:2.8796
좋고 눈물을 눈을_피해서 Bo_Peep Bo_Peep 타고 star 매번 넌 받아 친구와 아이처럼 모르게 그 떨리는 겪게 별로 내겐 A몰래 건가 SAY 아픈 날들 you cry 그래도 나는 믿어요 없어 찾아주고
00002400 step, loss:3.1470
세월 멋지게 지펴 그대 생각_하나되는 shake_it gold la_La 가슴이 억울한 Bo_Peep 맛 봐 이제는 you_dare 다_같이 누가 그댈 bad_men 오 you_da

00002860 step, loss:1.9100
일단 그냥 날 꽉_안아줘 대신 창문을 Bo_Peep 너와 큰_빈자리 또 너무나_많이 날 모르는_사람처럼 넌 놀랄지_몰라 너희 창문을 Bo_Peep 내가 잊혀질까 봐 더 뭘_어쩌겠어 아직도 고갤 숙인 채 아무_말도 하지 않고
00002870 step, loss:2.2594
You_make me smile oh_oh Let_me say Hey boy God 보는_거야 궁금한 거 살짝 눈빛에 돼 Hey boy 너의 말투 생각보다 너와의 바보같이 oh_oh oh_oh oh_oh oh_oh oh_oh oh_oh oh_oh oh_oh
00002880 step, loss:2.4034
fine_fine Come_On V_E di_diva 자꾸만 니_생각이 나 너의 믿어요 아무렇지_않은 척 난 내가_좋다고 버렸어 차갑게 말을 있는_건 나는 멈출_수 없어 날 바라 친구들은 약속해 내게 I_got 니_맘을 열 수_있는 hear_u
00002890 step, loss:2.7268
나를 떠나지_마요 Amazing 하지만 그녀 눈은 너무 불렀을 지금도 언제라도 감으면 환상에 빠져들어 그 바란 지쳐서 난 한없이 두근두근 가슴이 나보기가 이대로 영원토록 그_눈빛 순 잊지_말아줘 모습의 그 느낌을 나는
00002900 step, loss:2.6392
Be mercy Now keep this in mind before the singer off_the You're_my I_could and Fly_with be_your them_fool 날 기다릴거야 빠져_상상으로 너 널_위한 준비 아주_잠시만 by_day 조각난 수_없는데 때 만큼 있을꺼야
00002910 step, loss:2.8078
설레이는 내맘 You’re_so Time lalala_lalala 아무도 stuP zhende_zhende HOT HOT Cool 다다다 새로운_눈빛에 날_미치게 만들어 Ha 꼬리도 뜨거워요 미쳐 짜릿한 연애가 타고_흘러 되지는 나비 그 순간은 하나 되어 같